In [89]:
import pandas as pd
import re
from datetime import datetime as dt

YEAR_RANGE = (1900, 1950)

def remove(row):
    try:
        date = dt.strptime(row.Date, '%Y-%m-%d')
    except:
        return True
    
    if YEAR_RANGE and date.year >= YEAR_RANGE[0] and date.year <= YEAR_RANGE[1]:
        pass # in the range!
    else:
        return True

    if row['Exclude from visualization'] or row['Unsure whether drag artist']:
        return True
    
    no_city = row['City'] == ''
    no_performer = row['Performer'] == ''
    no_venue = row['Venue'] == ''
    unnamed_performer = 'unnamed' in row['Performer'].lower()
    
    if no_city and no_performer and no_venue:
        return True
    
    if unnamed_performer:
        return True

    return False

def extract_addresses_dict(normalized_df):
    addresses = {}
    rows_with_addresses = normalized_df[normalized_df['Address']!='']
    warnings = []
    for x in zip(rows_with_addresses['Date'], rows_with_addresses['Source'], rows_with_addresses['Venue'], rows_with_addresses['Address']):
        date, source, venue, address = x
        if venue == '':
            warnings.append(address)
        else:
            if not venue in addresses:
                addresses[venue] = {}
            if not source in addresses[venue]:
                addresses[venue][source] = address
    if len(warnings):
        print(f'Warning: {len(warnings)} Venues with no names have addresses:')
        print('- ' + '- '.join(warnings))

    return addresses

def reverse_comment_dict(comment_dict):
    comments_reverse = {}
    for performer, comments in comment_dict.items():
        if not performer in comments_reverse:
            comments_reverse[performer] = {}
        for source, comment in comments.items():
            if not comment in comments_reverse[performer]:
                comments_reverse[performer][comment] = []
            comments_reverse[performer][comment].append(source)
    return comments_reverse
        
def get_comments(df, comment_field='Comment on edge: revue', match_field='Revue', transform=None):
    comments = {}
    rows_with_comments = df[df[comment_field]!='']
    warnings = []
    for x in zip(rows_with_comments['Date'], rows_with_comments['Source'], rows_with_comments[match_field], rows_with_comments[comment_field]):
        date, source, match, comment = x
        comment = str(comment).strip()
        if transform:
            comment = transform(comment)
        if match == '':
            warnings.append(str(comment)[:40]+'...')
        else:
            if not match in comments:
                comments[match] = {}
            if not source in comments[match]:
                comments[match][source] = comment
    if len(warnings):
        print(f'Warning: {len(warnings)} mentions in `{comment_field}` with no value have comments:')
        print('- ' + '\n- '.join(warnings))

    return comments

def get_revue_comments_dict(df):
    return get_comments(df, 'Comment on edge: revue', 'Revue')

def get_performer_comments_dict(df):
    return get_comments(df, 'Comment on node: performer', 'Performer')

def get_venue_comments_dict(df):
    return get_comments(df, 'Comment on node: venue', 'Venue')

def get_city_comments_dict(df):
    return get_comments(df, 'Comment on node: city', 'City')

def get_true_value(row, type):
    if type == 'source':
        if row['Source clean'] != '':
            return row['Source clean']
        return row['Source']
    if type == 'performer':
        if row['Normalized performer'] != '':
            return row['Normalized performer']
        if row['Performer first-name'] != '' and row['Performer last-name'] != '':
            return row['Normalized performer']
        return row['Performer']
    if type == 'city':
        if row['Normalized City'] != '':
            return row['Normalized City']
        return row['City']
    if type == 'revue':
        if row['Normalized Revue Name'] != '':
            return row['Normalized Revue Name']
        return row['Revue name']
    if type == 'venue':
        if row['Normalized Venue'] != '':
            return row['Normalized Venue']
        return row['Venue']
    raise NotImplementedError(f'type `{type}` is not yet implemented')

def find_ref(row, eima=True):
    source = row.get('Source', '')
    source += ' ' + str(row.get('EIMA_ID', ''))
    source += ' ' + str(row.get('Newspapers_ID', ''))
    source += ' ' + str(row.get('EIMA', ''))
    source += ' ' + str(row.get('Search (newspapers.com)', ''))
    source += ' ' + str(row.get('Source clean', ''))
    
    is_eima = 'eima' in source.lower() or 'variety' in source.lower() or 'billboard' in source.lower()
    has_ref = re.search(r'(\d{7,10})', source)
    refs = list(set(re.findall(r'(\d{7,10})', source)))
    if has_ref and eima and is_eima:
        return '|'.join(refs)
    
    if has_ref and not eima and not is_eima:
        return '|'.join(refs)

    return ''
    

df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT0E0Y7txIa2pfBuusA1cd8X5OVhQ_D0qZC8D40KhTU3xB7McsPR2kuB7GH6ncmNT3nfjEYGbscOPp0/pub?gid=254069133&single=true&output=csv')
df = df.fillna('')
df = df.replace('—', '')
df = df.replace('—*', '')



# First, set up our references to EIMA and newspapers.com
df['EIMA'] = df.apply(lambda row: find_ref(row), axis=1)
df['Newspapers.com'] = df.apply(lambda row: find_ref(row, False), axis=1)

# Normalize dataframe
df['Source'] = df.apply(lambda row: get_true_value(row, 'source'), axis=1)
df['Venue'] = df.apply(lambda row: get_true_value(row, 'venue'), axis=1)
df['Performer'] = df.apply(lambda row: get_true_value(row, 'performer'), axis=1)
df['City'] = df.apply(lambda row: get_true_value(row, 'city'), axis=1)
df['Revue'] = df.apply(lambda row: get_true_value(row, 'revue'), axis=1)


# Drop filtered data
df['remove'] = df.apply(lambda row: remove(row), axis=1)
df = df.drop(df[df['remove']==True].index)

In [90]:
import requests
from tqdm.notebook import tqdm

In [91]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'
}

In [92]:
#from selenium import webdriver

In [93]:
#d = webdriver.Firefox()

In [94]:
#d.get('https://www.newspapers.com/api/browse/1//us/new-york/brooklyn/times-union_18390/1931/01/18/main-edition')

In [95]:
#d.get_cookies()

In [96]:
api_root = 'https://www.newspapers.com/api/browse/1/us/'

In [97]:
import json, jellyfish
from pathlib import Path

In [98]:
if not Path('newspapers-api/root.json').exists():
    r = requests.get(f'{api_root}', headers=headers)
    root_json = json.loads(r.text)
    if not Path('newspapers-api').exists():
        Path('newspapers-api').mkdir(parents=True)

    Path('newspapers-api/root.json').write_text(r.text)
    
root_json = json.loads(Path('newspapers-api/root.json').read_text())


In [99]:
# download all states

states = [x.get('name') for x in root_json.get('children')]
for state in tqdm(states):
    if not Path(f'newspapers-api/{state}').exists():
        Path(f'newspapers-api/{state}').mkdir(parents=True)
        
    if not Path(f'newspapers-api/{state}/root.json').exists():
        # print(f'---> downloading {state}')
        if r.status_code == 200:
            r = requests.get(f'{api_root}{state}/', headers=headers)
        else:
            exit()
        Path(f'newspapers-api/{state}/root.json').write_text(r.text)

In [12]:
states = [
    'new-york',
    'new-jersey',
    'california',
    'michigan',
    'indiana',
    'ohio',
    'pennsylvania',
    'massachusetts',
    'kansas',
    'texas',
    'kentucky'
]

In [100]:
# download all cities

download_states = states # ['new-york'] # restrict to states

cities = {x: [y.get('name') for y in json.loads(Path(f'newspapers-api/{x}/root.json').read_text()).get('children')] for x in states}

for state in [x for x in states if x in download_states]:
    for city in tqdm(cities[state], bar_format="{l_bar}{bar}{r_bar}", desc=state, leave=False):
        if not Path(f'newspapers-api/{state}/{city}').exists():
            Path(f'newspapers-api/{state}/{city}').mkdir(parents=True)
        
        if not Path(f'newspapers-api/{state}/{city}/root.json').exists():
            r = requests.get(f'{api_root}{state}/{city}', headers=headers)
            if r.status_code == 200:
                Path(f'newspapers-api/{state}/{city}/root.json').write_text(r.text)
            else:
                exit()
            
            
            
# download all newspapers per city

download_cities = ['albany']

state_counter = 0
t = tqdm(cities.items(), bar_format="{l_bar}{bar}{r_bar}", desc="hello", leave=False)
for state, cities in t:
    t.set_description(state)
    state_counter += 1
    city_counter = 0
    if not state in download_states:
        continue
    
    t2 = tqdm(cities, leave=False)
    for city in t2: # [x for x in cities if x in download_cities]:
        city_counter += 1
        t2.set_description(city)
        papers = [y.get('name') for y in json.loads(Path(f'newspapers-api/{state}/{city}/root.json').read_text()).get('children')]
        for paper in tqdm(papers, leave=False):
            search_path = f'{state}/{city}/{paper}'
            if not Path(f'newspapers-api/{search_path}').exists():
                Path(f'newspapers-api/{search_path}').mkdir(parents=True)

            if not Path(f'newspapers-api/{search_path}/root.json').exists():
                r = requests.get(f'{api_root}{search_path}', headers=headers)
                if r.status_code == 200:
                    Path(f'newspapers-api/{search_path}/root.json').write_text(r.text)
                else:
                    exit()
                

In [101]:
# find relevant papers
min_year = 1930
max_year = 1940

In [102]:
all_states = [x['name'] for x in json.loads(Path('newspapers-api/root.json').read_text()).get('children')]
all_cities = {x: [y.get('name') for y in json.loads(Path(f'newspapers-api/{x}/root.json').read_text()).get('children')] for x in states}
newspapers_in_range = {}

for state, cities in tqdm(all_cities.items()):
    # print(state)
    for city in cities:
        if Path(f'newspapers-api/{state}/{city}/root.json').exists():
            all_newspapers = [x['name'] for x in json.loads(Path(f'newspapers-api/{state}/{city}/root.json').read_text()).get('children')]
            for newspaper in all_newspapers:
                if Path(f'newspapers-api/{state}/{city}/{newspaper}/root.json').exists():
                    year_range = [int(x['name']) for x in json.loads(Path(f'newspapers-api/{state}/{city}/{newspaper}/root.json').read_text()).get('children')]
                    start_year = min(year_range)
                    end_year = max(year_range)
                    in_min_max = [x for x in year_range if x > min_year and x < max_year]
                    if in_min_max:
                        if not state in newspapers_in_range:
                            newspapers_in_range[state] = {}
                        if not city in newspapers_in_range[state]:
                            newspapers_in_range[state][city] = []
                            
                        newspapers_in_range[state][city].append((newspaper, in_min_max))

In [103]:
newspaper_list = list(set([re.sub(r' [A-Z][A-Z] ', ' ', x).lower() for x in df['Newspaper'] if x]))

for state, _ in tqdm(newspapers_in_range.items()):
    # print(f'processing {state}')
    for city, newspapers in _.items():
        for newspaper, years in newspapers:

            test = {cmp: jellyfish.jaro_winkler_similarity(newspaper, cmp) for cmp in newspaper_list if jellyfish.jaro_winkler_similarity(newspaper, cmp) > 0.9}
            
            if test:
                print(f'-----> downloading {newspaper} ({state}/{city})')
                for year in years:
                    if not Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/').exists():
                        Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}').mkdir(parents=True)

                    if not Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/root.json').exists():
                        request_url = f'{api_root}{state}/{city}/{newspaper}/{year}'
                        r = requests.get(request_url, headers=headers)
                        if r.status_code == 200:
                            Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/root.json').write_text(r.text)
                        else:
                            exit()
                    month_data = json.loads(Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/root.json').read_text())
                    months = [x['name'] for x in month_data.get('children')]
                    for month in months:
                        if not Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/{month}').exists():
                            Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/{month}').mkdir(parents=True)

                        if not Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/{month}/root.json').exists():
                            request_url = f'{api_root}{state}/{city}/{newspaper}/{year}/{month}'
                            r = requests.get(request_url, headers=headers)
                            if r.status_code == 200:
                                Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/{month}/root.json').write_text(r.text)
                            else:
                                exit()
            else:
                pass

-----> downloading russell-register_4987 (alabama/seale)
-----> downloading arizona-republic_208 (arizona/phoenix)
-----> downloading arizona-daily-star_4340 (arizona/tucson)
-----> downloading chico-record_25740 (california/chico)
-----> downloading los-angeles-evening-citizen-news_25896 (california/hollywood)
-----> downloading oakland-tribune_2 (california/oakland)
-----> downloading petaluma-argus-courier_4734 (california/petaluma)
-----> downloading salinas-morning-post_12325 (california/salinas)
-----> downloading santa-ana-register_3160 (california/santa-ana)
-----> downloading santa-rosa-republican_22046 (california/santa-rosa)
-----> downloading stockton-independent_21869 (california/stockton)
-----> downloading visalia-times-delta_3722 (california/visalia)
-----> downloading fort-collins-coloradoan_3769 (colorado/fort-collins)
-----> downloading hartford-courant_4465 (connecticut/hartford)
-----> downloading miami-tribune_901 (florida/miami)
-----> downloading pensacola-news-

In [104]:
state, city, newspaper, year

('wyoming', 'jackson', 'jacksons-hole-courier_5761', 1939)

In [52]:


request_url = f'{api_root}{state}/{city}/{newspaper}/{year}/08'
r = requests.get(request_url, headers=headers)


In [53]:
r.text

'{"node":{"date":"1939-08","type":"MONTH","name":"08"},"ancestors":[{"type":"YEAR","name":"1939"},{"id":5761,"type":"NEWSPAPER","name":"jacksons-hole-courier","displayName":"Jackson\'s Hole Courier"}],"children":[{"date":"1939-08-03","name":"03"},{"date":"1939-08-10","name":"10"},{"date":"1939-08-17","name":"17"},{"date":"1939-08-24","name":"24"},{"date":"1939-08-31","name":"31"}],"typeOfChildren":"DATE"}'

In [76]:
for newspaper, rows in df.groupby('Newspaper'):
    if len(rows) == len([x for x in rows['Newspaper_ID'] if x]):
        continue
    print(newspaper)


Albany NY Times-Union
Alexandria LA Town Talk
American Israelite
Amsterdam Evening Recorder
Atlanta Constitution
Bakersfield Californian
Baldwinsville Gazette and Farmers Journal
Baltimore Afro-American
Baltimore Sun
Billboard
Binghamton Press and Sun-Bulletin
Bloomington Pantagraph
Boston Globe
Brattleboro VT Reformer
Broadway Tattler
Brooklyn Chat
Brooklyn Citizen
Brooklyn Daily Eagle
Brooklyn Home Talk the Item
Brooklyn Life and Activities of Long Island Society
Brooklyn Standard Union
Brooklyn Times Union
Brownsville Herald
Buffalo Courier
Buffalo Evening News
Buffalo Times
Camden NJ Courier-Post
Camden NJ Morning Post
Caruthersville Journal
Cedar Rapids IA Gazette
Chicago Defender
Chicago World
Chico CA Enterprise
Chico CA Record
Cincinnati Enquirer
Cleveland Call and Post
Coast Advertiser
Cohoes American
Corpus Christi TX Caller-Times
Daily News
Daily Oklahoman
Daily Record
Daily Register
Davenport IA Daily Times
Davenport IA Democrat
Dayton OH Daily News
Dayton OH Herald
Des Mo